<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting-up-imports" data-toc-modified-id="Setting-up-imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting up imports</a></span></li><li><span><a href="#Setting-up-Constant-Hyperparameters" data-toc-modified-id="Setting-up-Constant-Hyperparameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting up Constant Hyperparameters</a></span></li><li><span><a href="#Setting-up-Parameters-and-Functions-for-Training" data-toc-modified-id="Setting-up-Parameters-and-Functions-for-Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setting up Parameters and Functions for Training</a></span><ul class="toc-item"><li><span><a href="#Hyperparameters-Search-Space" data-toc-modified-id="Hyperparameters-Search-Space-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Hyperparameters Search Space</a></span></li><li><span><a href="#Creating-the-training-function" data-toc-modified-id="Creating-the-training-function-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Creating the training function</a></span></li><li><span><a href="#Creating-the-evaluation-function" data-toc-modified-id="Creating-the-evaluation-function-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Creating the evaluation function</a></span></li></ul></li><li><span><a href="#Running-the-training" data-toc-modified-id="Running-the-training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running the training</a></span><ul class="toc-item"><li><span><a href="#Loading-data-for-training" data-toc-modified-id="Loading-data-for-training-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Loading data for training</a></span></li><li><span><a href="#Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm" data-toc-modified-id="Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Configuring the Tuner with a Scheduler and a Search Algorithm</a></span></li><li><span><a href="#Running-the-Tuner" data-toc-modified-id="Running-the-Tuner-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Running the Tuner</a></span></li></ul></li><li><span><a href="#Evaluating-the-best-Results" data-toc-modified-id="Evaluating-the-best-Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluating the best Results</a></span></li></ul></div>

# Setting up imports

In [1]:
import os
from itertools import product

import torch
from torch.nn import CrossEntropyLoss, Sequential
from torch.nn.functional import normalize
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision.transforms import CenterCrop, Resize, GaussianBlur
# from torchvision.transforms.functional import invert

import ray
from ray import tune
from ray.air import session, RunConfig, CheckpointConfig
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
# from ray.tune.search.hyperopt import HyperOptSearch
# from ray.tune.search.optuna import OptunaSearch
# from ray.tune.search import ConcurrencyLimiter


from dataset import POCDataReader, data_augment_, POCDataset
from metrics import Metrics, EvaluationMetrics
from models import UNet, DeepCrack, SubUNet
from loss import *
from pipelines import InputPipeline, SequenceFilters, SumFilters
from pipelines.filters import *
from train import training_loop, validation_loop
from train_tqdm import evaluation_loop


# Setting up Constant Hyperparameters

In [2]:
EPOCHS = 20
NUM_SAMPLES = 1

NUM_AUGMENT = 1

LOAD_DATA_ON_GPU = False
GPUS_PER_TRIAL = 1
CPUS_PER_TRIAL = 20

##### Selecting Cuda device

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Setting up Parameters and Functions for Training

## Hyperparameters Search Space

##### Preload Losses Functions

In [4]:
pixel_loss_list = [
    CrossEntropyLoss(weight=torch.tensor([.3, .7])),
    FocalLoss(weight=torch.tensor([.3, .7]), gamma=2),
]

volume_loss_list = [
    JaccardLoss(),
    TverskyLoss(alpha=0.3, beta=0.7),
    FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=2),
]

loss_list = []
for ploss in pixel_loss_list:
    loss_list.append(PixelLoss(pixel_loss=ploss, volume_loss=None))
for vloss in volume_loss_list:
    loss_list.append(VolumeLoss(pixel_loss=None, volume_loss=vloss))
for (ploss, vloss) in product(pixel_loss_list, volume_loss_list):
    loss_list.append(CombinedLoss(loss1=ploss, loss2=vloss, ratio=0.3))
    loss_list.append(BorderedLoss(border_loss=ploss, volume_loss=vloss, ratio=0.7))

##### Preload Filter

In [5]:
filter_list = [normalize] #, invert]

layer_list = [
    SobelFilter(),
    LaplacianFilter(),
    FrangiFilter(),
    SatoFilter(),
    SumFilters(FrangiFilter(), SatoFilter()),
    SkeletonFilter(SequenceFilters(SumFilters(FrangiFilter(), SatoFilter()), CrackBinaryFilter())),
]

pipeline_list = []
for f, l in product(filter_list, layer_list):
    pipeline_list.append(InputPipeline(transformer=f, layer_transformer=l))

##### Search Space

In [6]:
search_space = {
    "Network": UNet, #tune.grid_search([UNet, DeepCrack, SubUNet]),
    "Optimizer": Adam,

    "Learning Rate": 1e-4, #tune.loguniform(1e-6, 1e-4),
    "Batch Size": 4,           #tune.qrandint(2, 8, 2),

    "Loss Function": tune.grid_search(loss_list),

    "Negative Mining": False, #tune.choice([True, False]),
    "Smooth Labeling": False, #tune.choice([True, False]),

    "Input Pipeline": tune.grid_search(pipeline_list),
}

## Creating the training function

In [7]:
def train(config, train_data, val_data):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    inpip = config["Input Pipeline"]
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    train_dataset = POCDataset(
        train_data,
        transform=Sequential(inpip, CenterCrop(size=(480, 480)), Resize(size=(384, 384))),
        target_transform= Sequential(
            GaussianBlur(kernel_size=3, sigma=0.7),
            CenterCrop(size=(480, 480)),
            Resize(size=(384, 384)),
        ) if config["Smooth Labeling"] else Sequential(
            CenterCrop(size=(480, 480)),
            Resize(size=(384, 384)),
        ),
        negative_mining=config["Negative Mining"],
        load_on_gpu=LOAD_DATA_ON_GPU)
    train_dataset.precompute_transform()

    if LOAD_DATA_ON_GPU:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
        )
    else:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    val_dataset = POCDataset(
        val_data, 
        transform=Sequential(inpip, CenterCrop(size=(480, 480)), Resize(size=(384, 384))),
        target_transform=Sequential(CenterCrop(size=(480, 480)), Resize(size=(384, 384))),
        negative_mining=False,
        load_on_gpu=LOAD_DATA_ON_GPU)
    val_dataset.precompute_transform()
    
    if LOAD_DATA_ON_GPU:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True)
    else:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    model = config["Network"](n_channels=inpip.nb_channel, n_classes=2)
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    loss_fn = config["Loss Function"].to(device)
    optimizer = config["Optimizer"](model.parameters(), lr=config["Learning Rate"], betas=(0.9, 0.99))
    lr_scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state, scheduler_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        lr_scheduler.load_state_dict(scheduler_state)

    train_metrics = Metrics(
        buffer_size=len(training_dataloader),
        mode="Training",
        hyperparam=config,
        device=device)

    val_metrics = Metrics(
        buffer_size=len(validation_dataloader),
        mode="Validation",
        hyperparam=config,
        device=device)


    for epoch in range(1, EPOCHS+1):  # loop over the dataset multiple times
        training_loop(epoch, training_dataloader, model, loss_fn, optimizer, lr_scheduler, train_metrics, device)
        validation_loop(epoch, validation_dataloader, model, loss_fn, val_metrics, device)

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("model", exist_ok=True)
        torch.save((model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict()), "model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("model")
        session.report(metrics=val_metrics.get_metrics(epoch), checkpoint=checkpoint)

    train_metrics.close_tensorboard()
    val_metrics.close_tensorboard()


## Creating the evaluation function

In [8]:
def evaluate(test_data, result):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    inpip = result.config["Input Pipeline"]
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    test_dataset = POCDataset(
        test_data,
        transform=Sequential(inpip, CenterCrop(size=(480, 480)), Resize(size=(384, 384))),
        target_transform=Sequential(CenterCrop(size=(480, 480)), Resize(size=(384, 384))),
        negative_mining=False,
        load_on_gpu=LOAD_DATA_ON_GPU)
    
    if LOAD_DATA_ON_GPU:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
    else:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=2*CPUS_PER_TRIAL, pin_memory=True, pin_memory_device=device)

    best_trained_model = result.config["Network"](n_channels=inpip.nb_channel, n_classes=2).to(device)

    checkpoint_path = os.path.join(result.checkpoint.to_directory(), "checkpoint.pt")
    model_state, _, _ = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    test_metrics = EvaluationMetrics(
        buffer_size=len(evaluation_dataloader),
        hyperparam=result.config,
        epochs=result.metrics["Epoch"],
        device=device)

    evaluation_loop(dataloader=evaluation_dataloader, model=best_trained_model, metric=test_metrics, device=device)


# Running the training

## Loading data for training

In [9]:
data_reader = POCDataReader(root_dir="../data/POC", load_on_gpu=False, verbose=True)
train_data, val_data, test_data = data_reader.split([0.7, 0.1, 0.2])
data_augment_(train_data, n=NUM_AUGMENT, load_on_gpu=False, verbose=True)

Loading dataset into RAM:   0%|          | 0/2744 [00:00<?, ?it/s]

	- Loading done, RAM used: 4.79GiB / free: 50.15GiB / total: 125.40GiB
	- Got a total of 2744 images.


Expending the dataset 1 more times:   0%|          | 0/1920 [00:00<?, ?it/s]

	- Augmentation done, RAM used: 7.50GiB / free: 47.44GiB / total: 125.40GiB
	- Got 1920 new images and a total of 3840 images.


## Configuring the Tuner with a Scheduler and a Search Algorithm

In [10]:
# scheduler = ASHAScheduler(max_t=EPOCHS, grace_period=2, reduction_factor=2)
# search_algo = HyperOptSearch()
# search_algo = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="CrackIoU",
    mode="max",
    num_samples=NUM_SAMPLES,
#     scheduler=scheduler,
#     search_alg=search_algo,
)

tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train, train_data=train_data, val_data=val_data),
        resources={"cpu": CPUS_PER_TRIAL, "gpu": GPUS_PER_TRIAL}),
    tune_config=tune_config,
    param_space=search_space,
    run_config=RunConfig(
        local_dir="~/POC-Project/ray_results",
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="CrackIoU",
            checkpoint_score_order="max",
            checkpoint_at_end=False)))

## Running the Tuner

In [11]:
results = tuner.fit()

2023-04-17 15:26:15,075	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(train pid=6540) /home/pirl/anaconda3/envs/POC-env/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
(train pid=6540)   warnings.warn("Can't initialize NVML")
(train pid=6626) /home/pirl/anaconda3/envs/POC-env/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
(train pid=6626)   warnings.warn("Can't initialize NVML")
/home/pirl/anaconda3/envs/POC-env/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Trial name,CrackIoU,Epoch,Loss,MeanIoU,Tversky,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_c9562_00000,0.751568,20,0.0976049,0.873529,0.878531,2023-04-17_16-37-36,True,,451ddfe587004187888e63846894fb73,"0_Input_Pipeline=InputPipeline_normalize_SobelFilter,Loss_Function=PixelLoss_CrossEntropyLoss",pirl-PowerEdge-T640,20,141.223.108.122,6540,True,4263.6,203.895,4263.6,1681717056,0,,20,c9562_00000,0.0310352
train_c9562_00001,0.74039,20,0.0977082,0.867825,0.872052,2023-04-17_16-39-24,True,,90751b829a464815accdc3c110f64c56,"1_Input_Pipeline=InputPipeline_normalize_LaplacianFilter,Loss_Function=PixelLoss_CrossEntropyLoss",pirl-PowerEdge-T640,20,141.223.108.122,6626,True,4367.78,202.071,4367.78,1681717164,0,,20,c9562_00001,0.0304656
train_c9562_00002,0.748502,20,0.0976747,0.871922,0.877287,2023-04-17_18-05-40,True,,451ddfe587004187888e63846894fb73,"2_Input_Pipeline=InputPipeline_normalize_FrangiFilter,Loss_Function=PixelLoss_CrossEntropyLoss",pirl-PowerEdge-T640,20,141.223.108.122,6540,True,5284.11,210.351,5284.11,1681722340,0,,20,c9562_00002,0.0310352
train_c9562_00003,0.741304,20,0.0976705,0.868301,0.874002,2023-04-17_18-07-32,True,,90751b829a464815accdc3c110f64c56,"3_Input_Pipeline=InputPipeline_normalize_SatoFilter,Loss_Function=PixelLoss_CrossEntropyLoss",pirl-PowerEdge-T640,20,141.223.108.122,6626,True,5287.49,212.591,5287.49,1681722452,0,,20,c9562_00003,0.0304656
train_c9562_00004,0.746081,20,0.0977357,0.870606,0.877014,2023-04-17_19-52-48,True,,451ddfe587004187888e63846894fb73,"4_Input_Pipeline=InputPipeline_normalize_SumFilters_FrangiFilter_SatoFilter,Loss_Function=PixelLoss_CrossEntropyLoss",pirl-PowerEdge-T640,20,141.223.108.122,6540,True,6426.71,209.883,6426.71,1681728768,0,,20,c9562_00004,0.0310352
train_c9562_00005,0.737914,20,0.097692,0.866585,0.87197,2023-04-17_19-57-43,True,,90751b829a464815accdc3c110f64c56,"5_Input_Pipeline=InputPipeline_normalize_SkeletonFilter,Loss_Function=PixelLoss_CrossEntropyLoss",pirl-PowerEdge-T640,20,141.223.108.122,6626,True,6610.27,213.343,6610.27,1681729063,0,,20,c9562_00005,0.0304656
train_c9562_00006,0.741831,20,0.00767268,0.868565,0.865923,2023-04-17_21-05-19,True,,451ddfe587004187888e63846894fb73,"6_Input_Pipeline=InputPipeline_normalize_SobelFilter,Loss_Function=PixelLoss_FocalLoss",pirl-PowerEdge-T640,20,141.223.108.122,6540,True,4351.35,209.453,4351.35,1681733119,0,,20,c9562_00006,0.0310352
train_c9562_00007,0.736354,20,0.00769114,0.865794,0.862862,2023-04-17_21-15-25,True,,90751b829a464815accdc3c110f64c56,"7_Input_Pipeline=InputPipeline_normalize_LaplacianFilter,Loss_Function=PixelLoss_FocalLoss",pirl-PowerEdge-T640,20,141.223.108.122,6626,True,4661.64,208.924,4661.64,1681733725,0,,20,c9562_00007,0.0304656
train_c9562_00008,0.734131,20,0.00769451,0.864601,0.863835,2023-04-17_22-32-41,True,,451ddfe587004187888e63846894fb73,"8_Input_Pipeline=InputPipeline_normalize_FrangiFilter,Loss_Function=PixelLoss_FocalLoss",pirl-PowerEdge-T640,20,141.223.108.122,6540,True,5241.75,209.157,5241.75,1681738361,0,,20,c9562_00008,0.0310352
train_c9562_00009,0.728698,20,0.00769539,0.861852,0.859927,2023-04-17_22-44-08,True,,90751b829a464815accdc3c110f64c56,"9_Input_Pipeline=InputPipeline_normalize_SatoFilter,Loss_Function=PixelLoss_FocalLoss",pirl-PowerEdge-T640,20,141.223.108.122,6626,True,5323.17,211.71,5323.17,1681739048,0,,20,c9562_00009,0.0304656


2023-04-17 16:37:36,804	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SobelFilter
  )
), 'Loss Function': PixelLoss(
  (pixel_loss): CrossEntropyLoss()
)}
2023-04-17 16:39:24,994	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): LaplacianFilter
  )
), 'Loss Function': PixelLoss(
  (pixel_loss): CrossEntropyLoss()
)}
2023-04-17 18:05:41,208	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): FrangiFilter
  )
), 'Loss Function': PixelLoss(
  (pixel_loss): CrossEntropyLoss()
)}
2023-04-17 18:07:32,862	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Inpu

2023-04-18 14:18:50,557	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SkeletonFilter
  )
), 'Loss Function': VolumeLoss(
  (volume_loss): FocalTverskyLoss
)}
2023-04-18 15:02:07,537	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SobelFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): JaccardLoss
)}
2023-04-18 15:35:41,127	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): LaplacianFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): JaccardLoss
)}
2023-04-18 16:31:57,495	INFO tensorboardx.py:267 -- Removed the following hyperparamete

2023-04-19 10:10:25,760	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): LaplacianFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): FocalTverskyLoss
)}
2023-04-19 10:51:54,575	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): FrangiFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): FocalTverskyLoss
)}
2023-04-19 11:41:39,914	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SatoFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): CrossEntropyLoss()
  (loss2): FocalTverskyLoss
)}
2023-04-19 12:41:40,967	INFO tensorboardx.py:267 -- Re

2023-04-20 06:24:07,842	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SatoFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): FocalLoss
  (loss2): TverskyLoss
)}
2023-04-20 07:08:39,863	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): ('SumFilters', 'FrangiFilter', 'SatoFilter')
  )
), 'Loss Function': CombinedLoss(
  (loss1): FocalLoss
  (loss2): TverskyLoss
)}
2023-04-20 08:14:49,499	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): SkeletonFilter
  )
), 'Loss Function': CombinedLoss(
  (loss1): FocalLoss
  (loss2): TverskyLoss
)}
2023-04-20 08:24:33,017	INFO tensorboardx.py:267 -- Removed the f

# Evaluating the best Results

In [12]:
best_result = results.get_best_result(metric="CrackIoU", mode="max", scope="all")  # Get best result object
print("Best trial config: {}".format(best_result.config))
print("Best trial final validation loss: {}".format(best_result.metrics["Loss"]))
print("Best trial final validation CrackIoU: {}".format(best_result.metrics["CrackIoU"]))

for result in results:
    evaluate(test_data=test_data, result=result)

Best trial config: {'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Learning Rate': 0.0001, 'Batch Size': 4, 'Loss Function': BorderedLoss(
  (border_loss): CrossEntropyLoss()
  (volume_loss): JaccardLoss
), 'Negative Mining': False, 'Smooth Labeling': False, 'Input Pipeline': InputPipeline(
  (layer_transformer): ModuleList(
    (0): FrangiFilter
  )
)}
Best trial final validation loss: 0.08201704174280167
Best trial final validation CrackIoU: 0.771568238735199


[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

...

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]

[Evaluating]:   0%|          | 0/550 [00:00<?, ?it/s]